In [1]:
import pandas as pd
import pathlib
from sklearn import preprocessing
import numpy as np
import random

In [2]:
prices = pd.read_csv('../../data/854_1575_bundle_archive/prices-split-adjusted.csv')
prices

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
...,...,...,...,...,...,...,...
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0


In [3]:
len(np.unique(prices.symbol))

501

In [4]:
prices.date[0]

'2016-01-05'

In [5]:
prices["year"] = prices.date.str[:4]

In [6]:
prices.year

0         2016
1         2016
2         2016
3         2016
4         2016
          ... 
851259    2016
851260    2016
851261    2016
851262    2016
851263    2016
Name: year, Length: 851264, dtype: object

In [7]:
min(prices.date)

'2010-01-04'

In [8]:
details = pd.read_csv('../../data/854_1575_bundle_archive/securities.csv')
details[["Ticker symbol","GICS Sector","GICS Sub Industry"]]

,Ticker symbol,GICS Sector,GICS Sub Industry
0,MMM,Industrials,Industrial Conglomerates
1,ABT,Health Care,Health Care Equipment
2,ABBV,Health Care,Pharmaceuticals
3,ACN,Information Technology,IT Consulting & Other Services
4,ATVI,Information Technology,Home Entertainment Software
...,...,...,...
500,YHOO,Information Technology,Internet Software & Services
501,YUM,Consumer Discretionary,Restaurants
502,ZBH,Health Care,Health Care Equipment
503,ZION,Financials,Regional Banks


In [9]:
len(details["GICS Sector"].unique())

11

In [10]:
len(details["GICS Sub Industry"].unique())

124

In [11]:
fundamentals = pd.read_csv('../../data/854_1575_bundle_archive/fundamentals.csv')
fundamentals = fundamentals.drop(columns=fundamentals.columns[0]).dropna()
fundamentals["Key"] = fundamentals["Ticker Symbol"]+fundamentals["For Year"].astype(int).astype(str)
fundamentals = fundamentals.drop(columns="Ticker Symbol")
fundamentals = fundamentals.drop(columns="Period Ending")
fundamentals = fundamentals.drop(columns="For Year")
fundamentals

,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,Changes in Inventories,Common Stocks,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding,Key
0,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,0.0,127000000.0,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-3.670000e+08,-5.60,3.350000e+08,AAL2012
1,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,0.0,5000000.0,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.000000e+00,-11.25,1.630222e+08,AAL2013
2,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,0.0,7000000.0,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.000000e+00,4.02,7.169154e+08,AAL2014
3,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,0.0,6000000.0,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.000000e+00,11.39,6.681299e+08,AAL2015
4,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,5.981110e+08,-260298000.0,7000.0,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-2.709500e+07,5.29,7.328355e+07,AAP2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771,2.255000e+08,-40400000.0,-3.480000e+07,11.0,-3.423000e+08,4.330700e+09,166.0,1.083300e+09,-164600000.0,2700000.0,...,4.313300e+09,1.024000e+09,6.549900e+09,3.108100e+09,9.658000e+09,4.673300e+09,-6.183700e+09,4.26,1.690845e+08,ZBH2014
1772,4.320000e+08,-56100000.0,-2.750000e+07,1.0,-4.341000e+08,8.195300e+09,100.0,1.459300e+09,-205400000.0,3000000.0,...,5.862900e+09,1.617900e+09,9.887900e+09,1.733160e+10,2.721950e+10,5.997800e+09,-6.329100e+09,0.78,1.884615e+08,ZBH2015
1777,1.381000e+09,-99000000.0,9.000000e+06,54.0,-1.840000e+08,8.780000e+08,43.0,6.100000e+08,-178000000.0,5000000.0,...,3.357000e+09,1.415000e+09,9.400000e+08,5.618000e+09,6.558000e+09,4.561000e+09,0.000000e+00,1.01,4.990099e+08,ZTS2013
1778,1.071000e+09,69000000.0,-7.000000e+06,44.0,-1.800000e+08,9.580000e+08,81.0,8.820000e+08,-110000000.0,5000000.0,...,3.465000e+09,1.086000e+09,1.311000e+09,5.277000e+09,6.588000e+09,4.785000e+09,0.000000e+00,1.16,5.025862e+08,ZTS2014


In [12]:
fundamentals["Key"]

0       AAL2012
1       AAL2013
2       AAL2014
3       AAL2015
4       AAP2012
         ...   
1771    ZBH2014
1772    ZBH2015
1777    ZTS2013
1778    ZTS2014
1779    ZTS2015
Name: Key, Length: 1299, dtype: object

In [13]:
prices["Key"] = prices.symbol+prices.year
#prices = prices.drop(columns="symbol")
#prices = prices.drop(columns="date")


In [14]:
prices["Key"]

0         WLTW2016
1         WLTW2016
2         WLTW2016
3         WLTW2016
4         WLTW2016
            ...   
851259     ZBH2016
851260    ZION2016
851261     ZTS2016
851262     AIV2016
851263     FTV2016
Name: Key, Length: 851264, dtype: object

In [15]:
prices

,date,symbol,open,close,low,high,volume,year,Key
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0,2016,WLTW2016
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0,2016,WLTW2016
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0,2016,WLTW2016
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0,2016,WLTW2016
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0,2016,WLTW2016
...,...,...,...,...,...,...,...,...,...
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0,2016,ZBH2016
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0,2016,ZION2016
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0,2016,ZTS2016
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0,2016,AIV2016


In [16]:
data = prices.merge(fundamentals, on='Key')
data

,date,symbol,open,close,low,high,volume,year,Key,Accounts Payable,...,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding
0,2012-01-03,AAL,5.200000,5.120000,5.070000,5.220000,6105900.0,2012,AAL2012,3.068000e+09,...,2.351000e+10,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.60,3.350000e+08
1,2012-01-04,AAL,5.090000,5.030000,4.970000,5.180000,5268700.0,2012,AAL2012,3.068000e+09,...,2.351000e+10,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.60,3.350000e+08
2,2012-01-05,AAL,5.150000,5.470000,5.070000,5.480000,8216100.0,2012,AAL2012,3.068000e+09,...,2.351000e+10,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.60,3.350000e+08
3,2012-01-06,AAL,5.440000,5.600000,5.400000,5.670000,8323000.0,2012,AAL2012,3.068000e+09,...,2.351000e+10,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.60,3.350000e+08
4,2012-01-09,AAL,5.560000,5.720000,5.550000,5.800000,8029900.0,2012,AAL2012,3.068000e+09,...,2.351000e+10,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,-5.60,3.350000e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323947,2016-12-23,XLNX,60.380001,60.560001,60.080002,60.570000,1105700.0,2016,XLNX2016,3.072220e+08,...,4.823154e+09,3.918933e+09,9.460860e+08,2.589893e+09,2.233261e+09,4.823154e+09,2.213881e+09,0.0,2.14,2.574145e+08
323948,2016-12-27,XLNX,61.070000,61.240002,60.889999,61.580002,1594400.0,2016,XLNX2016,3.072220e+08,...,4.823154e+09,3.918933e+09,9.460860e+08,2.589893e+09,2.233261e+09,4.823154e+09,2.213881e+09,0.0,2.14,2.574145e+08
323949,2016-12-28,XLNX,62.139999,60.630001,60.590000,62.240002,2141000.0,2016,XLNX2016,3.072220e+08,...,4.823154e+09,3.918933e+09,9.460860e+08,2.589893e+09,2.233261e+09,4.823154e+09,2.213881e+09,0.0,2.14,2.574145e+08
323950,2016-12-29,XLNX,60.599998,60.740002,60.360001,61.119999,1398100.0,2016,XLNX2016,3.072220e+08,...,4.823154e+09,3.918933e+09,9.460860e+08,2.589893e+09,2.233261e+09,4.823154e+09,2.213881e+09,0.0,2.14,2.574145e+08


In [17]:
window_size = 32
run_lenght = 32
sampled_symbols = random.sample(list(data.symbol.unique()), 3)
data[data.symbol.isin(sampled_symbols)]
batch_data = []
for symbol in sampled_symbols:
    index = random.randint(window_size+run_lenght, len(data[data.symbol==symbol]))
    selected_data = data[data.symbol==symbol][index-(window_size+run_lenght):index]
    selected_data = selected_data.drop(["symbol","date","year","Key"],axis=1)
    batch_data.append(selected_data)   
batch_data[2]

,open,close,low,high,volume,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,...,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding
31663,43.169998,43.169998,42.630001,43.459999,588700.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
31664,43.340000,42.610001,42.490002,43.369999,1677100.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
31665,42.560001,42.180000,42.119999,42.619999,815100.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
31666,42.220001,42.029999,41.770000,42.340000,1008100.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
31667,41.810001,41.889999,41.610001,42.029999,559200.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31722,45.820000,46.070000,45.689999,46.139999,2651000.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
31723,45.889999,46.240002,45.779999,46.450001,1752000.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
31724,46.290001,47.660000,46.180000,47.660000,1495000.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08
31725,47.299999,47.799999,47.200001,47.889999,2471300.0,863700000.0,31200000.0,-73400000.0,2.0,-94500000.0,...,1.188270e+10,3.204300e+09,1.645100e+09,6.371100e+09,5.511600e+09,1.188270e+10,4.306800e+09,-315500000.0,-0.84,1.276190e+08


In [18]:
sampled_symbols = list(data.symbol.unique())
lens = []
for symbol in sampled_symbols:
    lens.append(len(data[data.symbol==symbol]))

In [19]:
min(lens)

126

In [20]:
fit_data = data.drop(["symbol","date","year","Key"], axis=1)
scaler = preprocessing.StandardScaler().fit(fit_data)
selected_data = scaler.transform(batch_data[0])
batch_data[0]

,open,close,low,high,volume,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,...,Total Assets,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,Earnings Per Share,Estimated Shares Outstanding
268326,229.020004,229.229996,228.500000,231.429993,1107800.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
268327,230.750000,231.289993,229.919998,231.639999,937300.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
268328,231.229996,231.270004,230.369995,231.899994,801700.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
268329,229.649994,229.110001,227.449997,230.259995,1635200.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
268330,229.210007,229.199997,227.789993,230.570007,808600.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268385,231.639999,233.830002,230.160004,233.860001,941000.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
268386,233.699997,234.440002,230.960007,235.190002,919400.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
268387,236.309998,235.020004,234.210007,236.860001,832300.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08
268388,234.419998,234.460007,233.259995,235.679993,955900.0,3.075500e+10,-2.821000e+09,63000000.0,18.0,-376000000.0,...,5.387000e+10,3.667000e+10,3.349700e+10,8.001000e+09,4.586900e+10,5.387000e+10,1.790450e+11,-9.956000e+09,6.37,2.317111e+08


In [21]:
batch_data[0].close.to_numpy()[2]

231.270004

In [22]:
returndata = list(map(lambda x: scaler.transform(x[0:4]),batch_data))

In [23]:
batch_data[0].iloc[[0]].close

268326    229.229996
Name: close, dtype: float64

In [24]:
unique_symbols = pd.unique(data.symbol)
sampled_symbols = np.random.choice(unique_symbols, int(len(unique_symbols)*0.2))
test_batch_data = []
window_size = 32
run_lenght = 32
for symbol in sampled_symbols:
    end_index = random.randint(window_size+run_lenght, len(data[data.symbol==symbol]))
    selected_data = data[data.symbol==symbol][end_index-(window_size+run_lenght):end_index]
    selected_data = selected_data.drop(["symbol","date","year","Key"], axis=1)
    test_batch_data.append(selected_data.astype('float32'))
test_batch_data    


[           open      close        low       high      volume  \
 7589  18.084999  18.084999  17.695000  18.405001  12573600.0   
 7590  17.855000  17.754999  17.725000  18.230000   5494000.0   
 7591  17.549999  17.610001  17.495001  17.885000   6307200.0   
 7592  17.645000  17.000000  16.889999  17.750000   9517200.0   
 7593  17.129999  17.350000  17.129999  18.334999  11045800.0   
 ...         ...        ...        ...        ...         ...   
 7648  20.820000  20.590000  20.275000  20.900000   5695800.0   
 7649  20.745001  20.395000  20.379999  21.004999   4141000.0   
 7650  20.665001  21.440001  20.530001  21.700001   5318600.0   
 7651  21.325001  21.184999  20.924999  21.430000   3981600.0   
 7652  21.150000  21.740000  20.990000  22.150000   6054000.0   
 
       Accounts Payable  Accounts Receivable  Add'l income/expense items  \
 7589       368947008.0          -58037000.0                  50635000.0   
 7590       368947008.0          -58037000.0                  5063

In [25]:
unique_symbols = pd.unique(data.symbol)
sampled_symbols = np.random.choice(unique_symbols, int(len(unique_symbols)*0.2), replace=False)

In [26]:
len(unique_symbols)

359

In [27]:
len(np.array([i for i in unique_symbols if i not in sampled_symbols]))

288

In [28]:
len(np.setdiff1d(unique_symbols, sampled_symbols))+len(sampled_symbols)

359

In [29]:
len(np.setdiff1d(unique_symbols, sampled_symbols))

288

In [30]:
np.random.choice(np.setdiff1d(unique_symbols, sampled_symbols), 32, replace=False)

array(['HAS', 'WYN', 'NEM', 'PAYX', 'CERN', 'FE', 'NTAP', 'MRO', 'ROST',
       'SWKS', 'MPC', 'FLR', 'ROP', 'MDLZ', 'CMS', 'SYMC', 'PVH', 'PEG',
       'HOG', 'XEC', 'ATVI', 'SYY', 'IRM', 'SCG', 'MCK', 'MKC', 'MJN',
       'KSS', 'GPS', 'VRSK', 'LVLT', 'R'], dtype=object)

In [31]:
len(sampled_symbols)

71

In [45]:
batch_data[0].to_numpy()

array([[ 2.29020004e+02,  2.29229996e+02,  2.28500000e+02, ...,
        -9.95600000e+09,  6.37000000e+00,  2.31711146e+08],
       [ 2.30750000e+02,  2.31289993e+02,  2.29919998e+02, ...,
        -9.95600000e+09,  6.37000000e+00,  2.31711146e+08],
       [ 2.31229996e+02,  2.31270004e+02,  2.30369995e+02, ...,
        -9.95600000e+09,  6.37000000e+00,  2.31711146e+08],
       ...,
       [ 2.36309998e+02,  2.35020004e+02,  2.34210007e+02, ...,
        -9.95600000e+09,  6.37000000e+00,  2.31711146e+08],
       [ 2.34419998e+02,  2.34460007e+02,  2.33259995e+02, ...,
        -9.95600000e+09,  6.37000000e+00,  2.31711146e+08],
       [ 2.35240005e+02,  2.34550003e+02,  2.33250000e+02, ...,
        -9.95600000e+09,  6.37000000e+00,  2.31711146e+08]])